## Setup and Import Libraries

In [2]:
import os
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.base import RunnableLambda
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
model = ChatOpenAI(model="gpt-3.5-turbo")

## Tagging

In [6]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [8]:
convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

In [9]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [10]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [11]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [12]:
tagging_chain = prompt | model_with_functions

In [13]:
response = tagging_chain.invoke({"input": "I love langchain"})
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"pos","language":"en"}', 'name': 'Tagging'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 108, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtwE661ZPOq7aPRZytBmIctglMls5', 'finish_reason': 'stop', 'logprobs': None}, id='run--3412757d-7446-4c96-a495-86fd239a24e2-0', usage_metadata={'input_tokens': 108, 'output_tokens': 10, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
response.additional_kwargs

{'function_call': {'arguments': '{"sentiment":"pos","language":"en"}',
  'name': 'Tagging'},
 'refusal': None}

In [15]:
response = tagging_chain.invoke({"input": "non mi piace questo cibo"})
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"it"}', 'name': 'Tagging'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtwE9Btnqleglur09U8RqRth222iZ', 'finish_reason': 'stop', 'logprobs': None}, id='run--4c1f5236-9c72-406b-9e4f-e180ef7023b6-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [16]:
response.additional_kwargs

{'function_call': {'arguments': '{"sentiment":"neg","language":"it"}',
  'name': 'Tagging'},
 'refusal': None}

In [17]:
tagging_chain_with_parser = prompt | model_with_functions | JsonOutputFunctionsParser()

In [18]:
response = tagging_chain_with_parser.invoke({"input": "non mi piace questo cibo"})
response

{'sentiment': 'neg', 'language': 'it'}

## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [19]:
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [20]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [21]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

In [22]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]

In [23]:
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [24]:
response = extraction_model.invoke("Joe is 30, his mom is Martha")
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 95, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtwELLf76RCVQpWvtHytxpYdJBNO6', 'finish_reason': 'stop', 'logprobs': None}, id='run--1085ba93-52c5-46fa-88be-e61fda9c71a9-0', usage_metadata={'input_tokens': 95, 'output_tokens': 21, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [25]:
response.additional_kwargs

{'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}',
  'name': 'Information'},
 'refusal': None}

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [27]:
extraction_chain = prompt | extraction_model

In [28]:
response = extraction_chain.invoke("Joe is 30, his mom is Martha")
response

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 112, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtwEOPd7zK5khEOi5OEbBkdmDcSNG', 'finish_reason': 'stop', 'logprobs': None}, id='run--4c75070d-3930-4187-b924-e9fd842ee258-0', usage_metadata={'input_tokens': 112, 'output_tokens': 21, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [29]:
response.additional_kwargs

{'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}',
  'name': 'Information'},
 'refusal': None}

In [30]:
extraction_chain_with_parser = prompt | extraction_model | JsonOutputFunctionsParser()

In [31]:
response = extraction_chain_with_parser.invoke({"input": "Joe is 30, his mom is Martha"})
response

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]}

In [32]:
extraction_chain_with_key_output_parser = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [33]:
response = extraction_chain_with_key_output_parser.invoke({"input": "Joe is 30, his mom is Martha"})
response

[{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]

## Doing it for real

We can apply tagging and extraction to a larger body of text.

In [34]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [35]:
doc = documents[0]

In [36]:
page_content = doc.page_content[:10000]

In [37]:
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.


In [38]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]

class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]

### Tagging

In [39]:
overview_tagging_function = [convert_pydantic_to_openai_function(Overview)]

In [40]:
tagging_model = model.bind(
    functions=overview_tagging_function,
    function_call={"name":"Overview"}
)

In [41]:
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [42]:
tagging_chain.invoke({"input": page_content})

{'summary': 'Overview of LLM powered autonomous agents focusing on planning, memory, and tool use components.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use'}

## Extraction

In [43]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]

In [44]:
extraction_model = model.bind(
    functions=paper_extraction_function, 
    function_call={"name":"Info"}
)

In [45]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [46]:
extraction_chain.invoke({"input": page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

### Fixing the Prompt

In [47]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article follow by its author. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""


prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [48]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [49]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)',
  'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al. 2023'}]

In [51]:
extraction_chain.invoke({"input": "hi"})

[]

### Applying Text Splitter

In [52]:
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [53]:
splits = text_splitter.split_text(doc.page_content)

In [54]:
len(splits)

15

In [55]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [56]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [57]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent 

In [58]:
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [59]:
prep.invoke("hi")

[{'input': 'hi'}]

In [60]:
chain = prep | extraction_chain.map() | flatten

In [62]:
chain.invoke(doc.page_content)

[{'title': 'AutoGPT', 'author': None},
 {'title': 'GPT-Engineer', 'author': None},
 {'title': 'BabyAGI', 'author': None},
 {'title': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)', 'author': None},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)', 'author': None},
 {'title': 'Duan et al. 2017', 'author': 'unknown'},
 {'title': 'Laskin et al. 2023', 'author': 'unknown'},
 {'title': 'Miller 1956', 'author': 'unknown'},
 {'title': 'LSH: Efficient Retrieval in High-Dimensional Space',
  'author': 'Andoni, Alexandr'},
 {'title': 'MRKL', 'author': 'Karpas et al. 2022'},
 {'title': 'TALM', 'author': 'Parisi et al. 2022'},
 {'title': 'Toolformer', 'author': 'Schick et al. 2023'},
 {'title': 'HuggingGPT', 'auth